In [1]:
import os
import numpy as np
import sys
from pathlib import Path

In [2]:
# for plotting
import matplotlib
# matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set()
# sns.set_style("whitegrid")
sns.set(font_scale=1.5, rc={'text.usetex' : True})
sns.set_context("paper", font_scale=4, rc={"lines.linewidth": 4})
mpl.rcParams['lines.linewidth'] = 3
plt.rcParams['axes.unicode_minus'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

<ipython-input-2-a4d10d561234>:17: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]


In [3]:
# import util functions
from util import *

# 1 Two-particle test setup
Particle $p_2$ is falling freely due to gravity $g = [0, -10, 0]$ in units of m/s$^2$ (i.e., gravity acts along downward direction). The initial velocity of $p_2$ is zero and the initial distance (boundary-to-boundary) is $d$ m, see Figure 1(a). In Figure 1(b), we show the equivalent setup where we start $p_2$ much closer to the $p_1$, with $\delta$ satisfying $0 \leq \delta < d$, and assign $p_2$ the initial velocity $v_0 = \sqrt{2 |g| \delta}$ that it would have acquired if it were falling freely $\delta$ distance due to the influence of gravity $g$.   

| <img src="assets/two-particles.png" width="400"> | 
| :---: | 
| **Figure 1.** Setup of a two-particle test |

To simulate this in PeriDEM, we need to do following:
- describe model setting such as
  * final time and time step
  * geometry of particles along with $d$ and $\delta$
  * discretization of two particles 
  * horizon (nonlocal length scale) in peridynamics
- material properties of two particles
- contact properties of two particles
- boundary conditions

Before we list the set of parameters, there is one key idea that we need to discuss. In the PeriDEM library, we allow arbitrary grouping of particles and walls. We refer to such groups by `zone`. 

## 1.1 Grouping particles - Zones
Suppose we are interested in multi-particle test. Consider following scenarios:
- **Case 1.** Suppose all particles have same shape. Suppose first $m$ particles, denoted by set $S_1 = \{1, 2, ..., m\}$, out of total $N$ particles have material properties of steel $M_1$ and remaining particles, in set $S_2 = \{m+1, m+2, ..., N\}$, of glass $M_2$. Suppose for all pair of particles, contact parameters are same.
- **Case 2.** Consider **case 1** but now also assume that
  * pair of particles in $S_1$ have contact properties $C_{11}$
  * pair of particles in $S_2$ have contact properties $C_{22}$
  * remaining pair of particles have contact properties $C_{12}$
- **Case 3.** Start from scratch and suppose now first $r$ particles, in set $\bar{S}_1 = \{1, 2, ..., r\}$, are of circular shape, and remaining particles, in set $\bar{S}_2 = \{r+1, r+2, ..., N\}$, are hexagon shaped. Also suppose all particles have same material properties and same contact propertis.
- **Case 4.** Let **case 3** holds. We now ask that some particles of set $\bar{S}_1$ have material properties $\bar{M}_{11}$ and some have material properties $\bar{M}_{12}$. Similarly, we ask that some subset of $\bar{S}_2$ have material properties $\bar{M}_{21}$ and remaining have $\bar{M}_{22}$
- **Case 5.** Let **case 1** holds. We now introduce a fixed wall to the particulate media. We assume that particles and walls have same material properties and same contact properties.
- **Case 6.** Start from scract, and assume that all particles are of same shape, same material properties, contact is same for any pair of particles. But now we assume that first $s$ particles, in set $\tilde{S}_1 = \{1, 2, ..., s\}$, are assigned a fine mesh, and remaining, in set $\tilde{S}_2 = \{s+1, s+2, ..., N\}$, are assigned a coarse mesh. 

The reason for outlining some of the possibility above is to show that in a multi-particle system, we often have to deal of subgroup specified properties. When properties change from one subset to another, it automatically generates a subgrouping. 

### 1.1.1 Case 1
Consider **case 1**. Here it is natural to create two zones (with zone we mean the subgroup); we will assign all particles in set $S_1$ the zone id $1$ and remaining particles zone id $2$. 

Once we have decided that simulation consists of two zones, we need to prescribe parameters such as
- Reference particle shape (see Remark 1 below on reference particle)
- Material properties
- Mesh of a reference particle (since all other particles are affine-transform related to reference particle, we only need mesh of a reference particle!)
- Contact defined for pair of zones
- Initial condition and boundary conditions (force or displacement) can either be applied by listing **global id** (i.e. 0, 1, 2, ..., N-1) or **zone id**. In the latter case, the conditions are applied to all particles in the specified **zone**.

> **Remark 1.** Reference particle is such that all particles are related by affine transformation of reference particle. You can think of finite element method where shape functions and their derivatives over any arbitrary element is computed using the map taking reference element to the arbitrary element. This reference element of course has a same shape as the arbitrary element.

For **case 1**, input file will have two zones and the properties as said above will be zone specific. Contact properties will be specified for the pair of zones, i.e., *<zone 1, zone 1>*, *<zone 2, zone 2>*, *<zone 1, zone 2>*. Note that contact properties are same for *<zone 1, zone 2>* and *<zone 2, zone 1>*. 

> **Remark 2.** When there is only one group (in situation when particles are of same shape, same material, same contact properties, and are obtained from same mesh of a reference particle), we will have just one **zone** in the simulation and all particles will belong to this zone.

### 1.1.2 Case 2
In this case, having just two zones (repeating that **zone** simply means subgroup) is enough as the contact in our simulation is always assigned for a pair of zones. So we can have the same input file as in **case 1** and simply modify the contact block of the input file where we can have properties $C_{11}, C_{22}, C_{12}$ for *<zone 1, zone 1>*, *<zone 2, zone 2>*, and *<zone 1, zone 2>* pairs.

### 1.1.3 Case 3
In this case, since the shape of particles differ and one group of particles defined by set $\bar{S}_1$ are of circular shape and remainig particles are of hexagon shape, it makes sense to again create two **zones**:
- add particles in set $\bar{S}_1$ to **zone 1**
- add remaining particles to **zone 2**.

We then specify the properties, as before, in a `zone-specific` fashion. We provide the details of reference particle (i.e., circle) and associated mesh in **zone 1** area of input file and do same of hexagon reference particle and associated mesh in **zone 2** area of input file. 

As before, we will define material properties for all particles in zone 1 and zone 2 and define the contact properties using pair of zones. We can have different material properties for particles in zone 1 and zone 2 and similary have different contact properties for different pair of two zones.

### 1.1.4 Case 4
In this case, having just two zones is not enough. If we look at **case 3**, we see that **case 4** further subdivides the **case 3** in the basis of material properties. We need **four zones** to handle this case:
- **zone 1.** circle shaped particles with material properties $\bar{M}_{11}$
- **zone 2.** circle shaped particles with material properties $\bar{M}_{12}$
- **zone 3.** hexagon shaped particles with material properties $\bar{M}_{21}$
- **zone 4.** hexagon shaped particles with material properties $\bar{M}_{22}$.

We then define the material properties, reference particle geometry and mesh in a `zone-specific` fashion and define the contact properties for pair of zones. In this case we have to specify 10 contact properties as there are 10 unique pairs of zones (we take *<i,j>* pair same as *<j,i>* for the purpose of contact).

### 1.1.5 Case 5
In our implementation, both particle and wall inherit from a base particle implementation, see [BaseParticle](https://github.com/prashjha/PeriDEM/blob/main/src/particle/baseParticle.h), however, conceptually both are treated differently. One of the major difference is as follows:

**Particle have associated reference particle so that they can be obtained by affine transformation of reference particle. However, for walls we do not assume or require to have reference particle. In the simulation, one has to describe all geometric details and mesh file for each wall seperately.**

Wall always belong to a different and unique zone. If we have n walls, we will atleast have n zones. If we have n walls and N particles, in the simplest case, we will have n+1 zone where n zones will correspond to n walls, and remaining zone will correspond to all the N particles. If particles are required to be grouped in smaller groups, say 4 groups, we will have n+4 zones.

Now that walls are assigned unique zones such that only one wall can be in one zone, and since the properties are specified in a `zone-specific` fashion, it is easy to specify properties of walls and group of particles in different zones. 

For the specific example in **case 5**, we will have 3 zones:
- **zone 1.** For the wall
- **zone 2.** For particles in set $S_1$ (see description of **case 1**)
- **zone 3.** For remaining particles.
We then have properties such as mesh, reference particle (only for zone 2 and 3), material properties for each zone, and have contact properties for pair of zones (in this case we need to assign contact properties for 6 unique pair of zones). 

### 1.1.6 Case 6
This example shows why concept of zone is handy. In this case, all particles are identical except their meshes. In this case it is natural to have two zones:
- **zone 1.** For particles with fine mesh
- **zone 2.** For particles with coarse mesh.

#  2 Preparation of input files for the two-particle test 
We now discuss the preparation of two-particle setup one by one.

## 2.1 Zones
Since we assume $p_1$ and $p_2$ have different material properties, we create two zones:
- **zone 1.** Particle 1 belongs to this zone
- **zone 2.** Particle 2 belongs to this zone.


## 2.2 Model setting
These are defined as follows:

In [4]:
## main units are meter and seconds

## set path for files created
fpath = './two_particles/'
Path(fpath).mkdir(parents=True, exist_ok=True)

## origin (this center is used to discretize the particle and
## later discretization is translated and rotated appropriately)
center = [0., 0., 0.]

## gravity
g = [0., -10., 0.]

## radius of p1 and p2
R1 = 0.0015
R2 = 0.001

## mesh size (use smaller radius to decide)
h = R1 / 8.
if R2 < R1:
    h = R2 / 8.
    
## peridynamics horizon (typically taken as 2 to 4 times the mesh size)
horizon = 3. * h


## initial distance between two particles
d = 0.001
# we take smallest delta while keeping p1 and p2 seperated by distance 'horizon'
delta = d - horizon 
v0 = [0, -np.sqrt(2. * np.abs(g[1]) * delta), 0.]

## final time and time step
T = 0.04
N_steps = 200000 
# if dt is large, the simulation could produce garbage results so care is needed
dt = T / N_steps 
# steps per output, i.e., simulation will generate output every 50 steps
N_out = N_steps / 50 

## 2.3 Material properties
The deformation of particles are governed by peridynamics model. To fully describe the properties of material, we need density $\rho$, bulk modulus $K$, Poisson ratio $\nu$, and critical energy release rate $G_c$. We define material properties of $p_1$ and $p_2$ below:

In [5]:
## material
# for zone 1 (in this case zone 1 has just one particle p1)
nu1 = 0.25
rho1 = 1200.
K1 = 2.16e+7
E1 = get_E(K1, nu1)  # see util.py
G1 = get_G(E1, nu1)
Gc1 = 50.

# for zone 2 (in this case zone 2 has just one particle p2)
nu2 = 0.25
rho2 = 1200.
K2 = 2.16e+7
E2 = get_E(K2, nu2)
G2 = get_G(E2, nu2)
Gc2 = 50.

## 2.4 Contact parameters
We now have to prescribe the contact parameters. As discussed earlier, contact is defined for pair of zones. In this case we have to assign contact properties for 3 pair of zones:

In [6]:
## contact radius is taken as R_contact_factor * h_{min} where
## h_{min} is the minimum mesh size over all meshes (it is computed at the begining 
## of simulation)
## we only specify the R_contact_factor
R_contact_factor = 0.95

## define Kn parameter for normal contact force (see PeriDEM paper for more details)
Kn_11 = 18. * get_eff_k(K1, K1) / (np.pi * np.power(horizon, 5))
Kn_22 = 18. * get_eff_k(K2, K2) / (np.pi * np.power(horizon, 5))
Kn_12 = 18. * get_eff_k(K1, K2) / (np.pi * np.power(horizon, 5))

## define beta_n parameter for normal damping
beta_n_eps = 0.9
# factor for damping (Constant C in the PeriDEM paper)
beta_n_factor = 100.
damping_active = True 

## friction coefficient
friction_coeff = 0.5
friction_active = False # friction is not active

## 2.5 Particle locations
We create a `.csv` file that lists out the `zone id` (0-numbered), `radius`, `center location` (x, y, z), `orientation` (the initial confguration is obtained by rotating the particle mesh by amount specified in `orientation`). 

Function below does this job:

In [7]:
def two_particle_locations(filename, radius, centers, pp_tag = None):
    """
    Creates .csv file containing the center, radius, orientation, and zone id of each particle in two-particle setup

    Parameters
    ----------
    filename : str
      Filename of .csv file to be created
    radius: list
      List containing radius of two particles
    centers: list of list
      List of list containing the centers of two particles
    pp_tag: str, optional
      Postfix .geo file with this tag
    """
    pp_tag_str = '_{}'.format(str(pp_tag)) if pp_tag is not None else ''
    inpf = open(filename + pp_tag_str + '.csv','w')
    inpf.write('i, x, y, z, r, o\n')
    for i in range(2):
        inpf.write('%d, ' % i)
        inpf.write(print_list(centers[i], '%Lf'))
        inpf.write(', %Lf' % radius[i])
        o = 0. if i == 0 else np.pi * 0.5
        inpf.write(', %Lf\n' % o)
    inpf.close()

In [8]:
# create particle_locations.csv file 
centers = []
centers.append([R1, R1, 0.])
centers.append([R1, 2.*R1 + R2 + d - delta, 0.])
two_particle_locations(fpath + 'particle_locations', [R1, R2], centers)

## 2.6 Discretization of reference particle
We have two zones with each containing single circular particle. We need to define reference particles for both zones and provide mesh file. 

We let both reference particle (circle in this case) be at center $[0,0,0]$. Radius of reference particle for zone 1 is same as radius of particle $p_1$, i.e., $R_1$. Similar is true for other reference particle.

We use `Gmsh` to create the mesh of particles. First step to mesh is to create a `.geo` file that `Gmsh` understands. This is done next:

In [9]:
## file below creates .geo file (see util.py)
# we need to provide the name of file (function will add .geo suffix)
# and also center, radius, and mesh size
# center is the center of reference particle. In the simulation, reference particle will affinely transformed
# to get the actual particle.
# Note center is defined as [0., 0., 0.] somewhere above
generate_circle_gmsh_input(fpath + 'p1', center, R1, h) # for p1
generate_circle_gmsh_input(fpath + 'p2', center, R2, h) # for p2

We can now discretize the circles using `.geo` file as follows

In [10]:
fp = fpath + 'p1'
# create .msh file
!gmsh $fp'.geo' -2 -o $fp'.msh' &> /dev/null
# .vtk is not required. But it may allow you to check if mesh looks okay
!gmsh $fp'.geo' -2 -o $fp'.vtk' &> /dev/null  

In [11]:
fp = fpath + 'p2'
# create .msh file
!gmsh $fp'.geo' -2 -o $fp'.msh' &> /dev/null
# .vtk is not required. But it may allow you to check if mesh looks okay
!gmsh $fp'.geo' -2 -o $fp'.vtk' &> /dev/null  

## 2.7 Putting all together
We now have defined and obtained all components for the simulation and we only need to create a instruction file `<your filename>.yaml`. 

We do this in one go below:

In [12]:
## open a input file (we choose pretty obvious name input.yaml)
inpf = open(fpath + 'input.yaml','w')

## provide model specific details
inpf.write("Model:\n")
inpf.write("  Dimension: 2\n")
inpf.write("  Discretization_Type:\n")
inpf.write("    Spatial: finite_difference\n") # we refer to meshfree discretization by finite_difference
inpf.write("    Time: central_difference\n")   # you can use either central_difference or velocity_verlet
inpf.write("  Final_Time: %4.6e\n" % (T))
inpf.write("  Time_Steps: %d\n" % (N_steps))

## not used (I should get rid of this soon)
inpf.write("Policy:\n")
inpf.write("  Enable_PostProcessing: true\n")

## container info
## Container is the region in 2d (or in 3d) where you expect all activites 
## to take place. If not sure, you can take a wild guess of bounding box of
## this simulation.
inpf.write("Container:\n")
inpf.write("  Geometry:\n")
inpf.write("    Type: rectangle\n")
# we define container to be rectangle. 
# in the code, we need to provide 6 element vector to define a rectangle
# first 3 in the vector correspond to coordinate of the left-bottom corner
# and remaining correspond to coordinates of the right-top corner
contain_params = [0., 0., 0., 2.*R1, 2.*R1 + 2.*R2 + d, 0.]
if R2 > R1:
    contain_params[3] = 2.*R2
inpf.write("    Parameters: " + print_dbl_list(contain_params))

## zone info
## we have two zones
inpf.write("Zone:\n")
inpf.write("  Zones: 2\n")

## zone 1 (bottom particle)
inpf.write("  Zone_1:\n")
# specify that this is not a zone containing wall
inpf.write("    Is_Wall: false\n")

## zone 2 (top particle)
inpf.write("  Zone_2:\n")
inpf.write("    Is_Wall: false\n")

## particle info
## as we said earlier, the properites are specific in a `zone-specific` manner
## what I really mean by `zone-specific` will be clear now
inpf.write("Particle:\n")
# for two-particle test, we not only output the standard .vtu simulation file
# but also some other quantities such as maximum distance between two particles
# this is to debug the code and to account for the damping effects
inpf.write("  Test_Name: two_particle\n")
# specify reference particle details for zone 1 particles (in this case zone 1 has just one particle)
inpf.write("  Zone_1:\n")
# reference particle is of circular type
inpf.write("    Type: circle\n")
# in the code, circle is defined using 4 vector array
# first element in the vector is radius, and rest are the coordinates of the center
# recall that we fixed referene particles at center = [0,0,0]
p1_geom = [R1, center[0], center[1], center[2]]
inpf.write("    Parameters: " + print_dbl_list(p1_geom)) 
inpf.write("  Zone_2:\n")
# zone 2 reference particle is also circle. The radius is different and center is same.
inpf.write("    Type: circle\n")
p2_geom = [R2, center[0], center[1], center[2]]
inpf.write("    Parameters: " + print_dbl_list(p2_geom)) 

## particle generation
## We generate particles by reading the particle_locations.csv file which provides list of particles
## in the simulation (each row correspond to unique particle)
## for each particle, particle_locations.csv provides information such as 
## <zone id>, <center>, <radius>, <orientation>
inpf.write("Particle_Generation:\n")
inpf.write("  From_File: particle_locations.csv\n")
# loc_rad_orient basically says we provide center location, radiua, and orientation
inpf.write("  File_Data_Type: loc_rad_orient\n") 

## Mesh info
inpf.write("Mesh:\n")

# zone 1
inpf.write("  Zone_1:\n")
inpf.write("    File: p1.msh \n")

# zone 2
inpf.write("  Zone_2:\n")
inpf.write("    File: p2.msh \n")

## Contact info
## we have to provide contact parameters for pair of zones
inpf.write("Contact:\n")

# 11
# this fixes the contact between two particles in zone 1
inpf.write("  Zone_11:\n")
inpf.write("    Contact_Radius_Factor: %4.6e\n" % (R_contact_factor))

if damping_active == False:
    inpf.write("    Damping_On: false\n")
if friction_active == False:
    inpf.write("    Friction_On: false\n")

inpf.write("    Kn: %4.6e\n" % (Kn_11))
inpf.write("    Epsilon: %4.6e\n" % (beta_n_eps))
inpf.write("    Friction_Coeff: %4.6e\n" % (friction_coeff))
inpf.write("    Kn_Factor: 1.0\n")
inpf.write("    Beta_n_Factor: %4.6e\n" % (beta_n_factor))

# 12
# this fixes the contact between two particles in zone 1 and zone 2
inpf.write("  Zone_12:\n")
inpf.write("    Contact_Radius_Factor: %4.6e\n" % (R_contact_factor))

if damping_active == False:
    inpf.write("    Damping_On: false\n")
if friction_active == False:
    inpf.write("    Friction_On: false\n")

inpf.write("    Kn: %4.6e\n" % (Kn_12))
inpf.write("    Epsilon: %4.6e\n" % (beta_n_eps))
inpf.write("    Friction_Coeff: %4.6e\n" % (friction_coeff))
inpf.write("    Kn_Factor: 1.0\n")
inpf.write("    Beta_n_Factor: %4.6e\n" % (beta_n_factor))

# 22
inpf.write("  Zone_22:\n")
inpf.write("    Contact_Radius_Factor: %4.6e\n" % (R_contact_factor))

if damping_active == False:
    inpf.write("    Damping_On: false\n")
if friction_active == False:
    inpf.write("    Friction_On: false\n")

inpf.write("    Kn: %4.6e\n" % (Kn_22))
inpf.write("    Epsilon: %4.6e\n" % (beta_n_eps))
inpf.write("    Friction_Coeff: %4.6e\n" % (friction_coeff))
inpf.write("    Kn_Factor: 1.0\n")
inpf.write("    Beta_n_Factor: %4.6e\n" % (beta_n_factor))

## Neighbor info
## at present, this block has no impact in the simulation. We have it for later extensions of the library.
inpf.write("Neighbor:\n")
inpf.write("  Update_Criteria: simple_all\n")
inpf.write("  Search_Factor: 5.0\n")

## Material info
## material properties have to be specified in a `zone-specific` manner
inpf.write("Material:\n")

# zone 1
inpf.write("  Zone_1:\n")
inpf.write("    Type: PDState\n")
inpf.write("    Horizon: %4.6e\n" % (horizon))
inpf.write("    Density: %4.6e\n" % (rho1))
inpf.write("    Compute_From_Classical: true\n")
inpf.write("    K: %4.6e\n" % (K1))
inpf.write("    G: %4.6e\n" % (G1))
inpf.write("    Gc: %4.6e\n" % (Gc1))
inpf.write("    Influence_Function:\n")
inpf.write("      Type: 1\n")

# zone 2
inpf.write("  Zone_2:\n")
inpf.write("    Type: PDState\n")
inpf.write("    Horizon: %4.6e\n" % (horizon))
inpf.write("    Density: %4.6e\n" % (rho2))
inpf.write("    Compute_From_Classical: true\n")
inpf.write("    K: %4.6e\n" % (K2))
inpf.write("    G: %4.6e\n" % (G2))
inpf.write("    Gc: %4.6e\n" % (Gc2))
inpf.write("    Influence_Function:\n")
inpf.write("      Type: 1\n")

## Force
# `Gravity` force is of body force type and all particles and walls experience this force
inpf.write("Force_BC:\n")
inpf.write("  Gravity: " + print_dbl_list(g))

## IC
## Initial condition and boundary conditions can be applied in many ways
## method 1: apply conditions on all particles in particular zone in which case one has to provide the 
##           Zone_List
## method 2: apply conditions on specific particles by providing their global id. This is the current case.
##           In Particle_List we prove `1` and say that apply constant velocity initial condition v0
##           to particle 1. Particles are numbered starting from 0, so 1 correspons to p2 and 0 corresponds to 
##           p1
## method 3: apply conditions on any particle withing specified region. So one can specify a rectangular region
##           and apply conditions on any particle that falls in that region.
inpf.write("IC:\n")
inpf.write("  Constant_Velocity:\n")
inpf.write("    Velocity_Vector: " + print_dbl_list(v0))
inpf.write("    Particle_List: [1]\n")

## Displacement
inpf.write("Displacement_BC:\n")
# it is typical to have many different type of boundary conditions so it is conventient to 
# consider a sets of boundary condition (force boundary condition is similar)
# in this case we just have 1 set
inpf.write("  Sets: 1\n")
inpf.write("  Set_1:\n")
# this set is applicable to `0` particle (bottom)
inpf.write("    Particle_List: [0]\n")
# displacement components along x and y are impacted
inpf.write("    Direction: [1,2]\n")
# displacement is fixed by a function which is constant in time and space. Further, the value of this 
# constant is 0 (see Parameters block which expects single parameter for constant type function)
inpf.write("    Time_Function:\n")
inpf.write("      Type: constant\n")
inpf.write("      Parameters:\n")
inpf.write("        - 0.0\n")
inpf.write("    Spatial_Function:\n")
inpf.write("      Type: constant\n")
# for `0` particle we have fixed all degree of freedoms and the displacement is fixed to 0
# so we explicitly say that for this particle all defs are zero
# this extra information allows us to not compute this boundary condition every time step as there is nothing
# really to compute (displacemnts are always fixed to zero)
inpf.write("    Zero_Displacement: true\n")

## Output info
## in this block, we take care of simulation output
inpf.write("Output:\n")
# where to write files
inpf.write("  Path: %s\n" %(fpath))
inpf.write("  Tags:\n")
# list the variable that you want to output in the .vtu file
inpf.write("    - Displacement\n")
inpf.write("    - Velocity\n")
inpf.write("    - Force\n")
inpf.write("    - Force_Density\n")
inpf.write("    - Damage_Z\n")
inpf.write("    - Damage\n")
inpf.write("    - Nodal_Volume\n")
inpf.write("    - Zone_ID\n")
inpf.write("    - Particle_ID\n")
inpf.write("    - Fixity\n")
inpf.write("    - Force_Fixity\n")
inpf.write("    - Contact_Nodes\n")
inpf.write("    - No_Fail_Node\n")
inpf.write("    - Boundary_Node_Flag\n")
inpf.write("    - Theta\n")
# how frequent the output should take place
inpf.write("  Output_Interval: %d\n" % (N_out))
inpf.write("  Compress_Type: zlib\n")
inpf.write("  Perform_FE_Out: false\n")
inpf.write("  Perform_Out: true\n")
# how frequent we dump the test specific outputs
inpf.write("  Test_Output_Interval: %d\n" % (N_out))
# what is verbosity level (last time I checked, the maximum is 3 so if you have 3 or above it will
# produce maximum debug information)
inpf.write("  Debug: 3\n")
# assign this simulation a tag
inpf.write("  Tag_PP: 0\n")

## close file
## damn I feel tired already!!
inpf.close()

## 3 Running the simulation
By now, we should have following files
- `particle_locations.csv`
- `p1.msh` and `p2.msh`
- `input.yaml` .

We run the simulation as follows:

In [ ]:
PeriDEM = <path to peridem executible>/PeriDEM
Nthreads = 4
# comment out below once path to peridem is set correctly
#!$PeriDEM -i input.yaml -nThreads $Nthreads